## **1. Loading libraries and data**

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ai-robotics-employee-salary/train_dataset.csv
/kaggle/input/ai-robotics-employee-salary/test_dataset.csv
/kaggle/input/ai-robotics-employee-salary/train_salaries.csv


In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats